In [9]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /Users/egor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def tag1(string):
    a = morph.parse(string)[0].tag
    return (('ADJF' in a))

def tag2(string):
    a = morph.parse(string)[0].tag
    return (('ADVB' in a))

def tag3(string):
    a = morph.parse(string)[0].tag
    return (('ADJS' in a))

def tag4(string):
    a = morph.parse(string)[0].tag
    return (('COMP' in a))

def tag5(string):
    a = morph.parse(string)[0].tag
    return (('NOUN' in a))

def tag6(string):
    a = morph.parse(string)[0].tag
    return (('INFN' in a))


#('NOUN' in a) or 

In [11]:
df = pd.read_csv('rang_train.csv')
df.head()

,id,title,text,score
0,0,просьба о отмена штраф каско,10 06 2016г я быть заключить договор №с041...,1
1,1,в целое хорошо работать по ипотека,брать ипотека в юникора порадовать квалифика...,0
2,2,мы всегда выручать банк уралсиб,знакомство с банк начаться с ипотека в 2009 го...,1
3,3,абсолютно неприемлемый ситуация и она решение,клиент совкомбанк с 2017 год в февраль 2018 г...,1
4,4,внести не туда,прийти оплачивать кредит и ошибочно внести ден...,1


In [12]:
df['text'] = df.apply(lambda x: x['title'] + ' ' + x['text'], axis=1)
df = df.drop('title', 1)

In [13]:
df['score'].value_counts()

1    7425
0    2575
Name: score, dtype: int64

In [14]:
good_words = pd.Series(np.concatenate(df[df['score']==0]['text'].apply(
    lambda s: s.strip().split()).values))
bad_words = pd.Series(np.concatenate(df[df['score']==1]['text'].apply(
    lambda s: s.strip().split()).values))

In [15]:
print(len(good_words), len(bad_words))

471708 840654


In [16]:
d = {}
for word in good_words.values:
    if word in d:
        d[word] += 1
    else:
        d[word] = 1
        
good_words = sorted(d, key = lambda x: d[x], reverse=True)

d = {}
for word in bad_words.values:
    if word in d:
        d[word] += 1
    else:
        d[word] = 1
        
bad_words = sorted(d, key = lambda x: d[x], reverse=True)

In [17]:
good_words = pd.Series(good_words)
bad_words = pd.Series(bad_words)

In [18]:
good_words = good_words[~good_words.isin(stopwords.words('russian'))]
bad_words = bad_words[~bad_words.isin(stopwords.words('russian'))]

In [19]:
print(len(good_words), len(bad_words))

19983 27186


In [20]:
good_words = good_words[good_words.str.isalpha()] # Только буквы
bad_words = bad_words[bad_words.str.isalpha()] # Только буквы

In [21]:
print(len(good_words), len(bad_words))

14817 20489


In [22]:
good_words = good_words[(good_words.str.len() >= 3)] # Длина > 2
bad_words = bad_words[(bad_words.str.len() >= 3)] # Длина > 2

In [23]:
print(len(good_words), len(bad_words))

14613 20222


# Топы частей речи

In [24]:
good_words1 = good_words[good_words.apply(tag1)]
bad_words1 = bad_words[bad_words.apply(tag1)]
good_words2 = good_words[good_words.apply(tag2)]
bad_words2 = bad_words[bad_words.apply(tag2)]
good_words3 = good_words[good_words.apply(tag3)]
bad_words3 = bad_words[bad_words.apply(tag3)]
good_words4 = good_words[good_words.apply(tag4)]
bad_words4 = bad_words[bad_words.apply(tag4)]
good_words5 = good_words[good_words.apply(tag5)]
bad_words5 = bad_words[bad_words.apply(tag5)]
good_words6 = good_words[good_words.apply(tag6)]
bad_words6 = bad_words[bad_words.apply(tag6)]

In [57]:
x1 = list(good_words1.values[:200])


In [58]:
y1 = list(bad_words1.values[:200])
len(y1)

200

In [59]:
x2 = list(good_words2.values[:200])
len(x2)

200

In [60]:
y2 = list(bad_words2.values[:200])
len(y2)

200

In [61]:
x3 = list(good_words3.values[:200])
len(x3)

37

In [62]:
y3 = list(bad_words3.values[:200])
len(y3)

70

In [63]:
y4 = list(bad_words4.values[:200])
len(y4)

6

In [64]:
x5 = list(good_words5.values[:200])
len(x5)

200

In [65]:
y5 = list(bad_words5.values[:200])
len(y1)

200

In [66]:
x6 = list(good_words6.values[:200])
len(x6)

200

In [67]:
y6 = list(bad_words6.values[:200])
len(y6)

200

In [68]:
bag = set()
bag = bag.union(set(x1), set(x2), set(x3), set(x5), set(x6), set(y1), set(y2), set(y3), set(y4), set(y5), set(y6))

In [69]:
bag = list(bag)
len(bag)

1075

In [30]:
sub = pd.DataFrame(bag) 


In [31]:
sub.to_csv('rang_words60.csv', index=False)

In [25]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [470]:
x1 = list(good_words1.values[:121]) # прилагательные 200 200
y1 = list(bad_words1.values[:125])
x2 = list(good_words2.values[:105]) # наречие 200 200
y2 = list(bad_words2.values[:100])
x3 = list(good_words3.values[:40]) # краткие прилагательные 37 70
y3 = list(bad_words3.values)[:45]
y4 = list(bad_words4.values[:5]) # лучше, хуже 6
x5 = list(good_words5.values[:100]) # существ 200 200
y5 = list(bad_words5.values[:40])
x6 = list(good_words6.values[:100]) # глаголы 200 200
y6 = list(bad_words6.values[:105])
s = set()
s = s.union(set(x1), set(x2), set(x3), set(x5), set(x6), set(y1), set(y2), set(y3), set(y4), set(y5), set(y6))
len(s)

In [471]:
words = list(s)

In [472]:
df = pd.read_csv('rang_train.csv')
df['text'] = df.apply(lambda x: x['title'] + ' ' + x['text'], axis=1)
for i in words:
    df[i] = df['text'].apply(lambda x: int(i in x))
df = df.drop('title', 1)
df = df.drop('text', 1)
df = df.drop('id', 1)
X = df.drop('score', axis=1)
y = df['score']

In [473]:
logistic = LogisticRegression()
llogistic4 = LogisticRegression()
np.mean(cross_val_score(logistic4, X, y, cv=5))

0.8545

# Топ слов без пересечения

In [149]:
good = good_words[:400]
bad = bad_words[:400]

In [150]:
un = set()
per = set()

In [151]:
per = set(good).intersection(set(bad))

In [152]:
len(per)

296

In [153]:
un = set(good).union(set(bad))

In [154]:
len(un)

504

In [155]:
un.difference_update(per)

In [156]:
len(un)

208

In [157]:
un = list(un)
sub = pd.DataFrame(un) 
sub.to_csv('no_per3.csv', index=False)

# Дамир

In [40]:
pos = pd.read_csv('words_positive')
neg = pd.read_csv('words_negative')
pos = set(pos['word'])
neg = set(neg['word'])
words = ()
words = list(pos.union(neg))
len(words)

781

In [41]:
sub = pd.DataFrame(words) 
sub.to_csv('no_per_damir.csv', index=False)

In [42]:
pos = pd.read_csv('final_words_pos.csv')
neg = pd.read_csv('final_words_neg.csv')
pos = set(pos['word'])
neg = set(neg['word'])
words = ()
words = list(pos.union(neg))
len(words)

369

In [43]:
sub = pd.DataFrame(words) 
sub.to_csv('no_per_damir2.csv', index=False)